## <center>  MATH11146: Modern Optimization Methods for Big Data Problems </center>

<center> University of Edinburgh</center>

<center>Lecturer: Peter Richtarik</center>

<center>Tutors: Sona Galovicova, Filip Hanzely and Nicolas Loizou</center>

##  <center>Lab 2: Introduction to Gradient descent methods</center>
<center>(C) Filip Hanzely and Peter Richtarik </center>
<center> 18.1.2017 </center>


## 1. Unicode characters in Julia

Julia supports unicode characters. For instance, this means that we can use some of the LaTeX symbols as variables. Example: Type \beta and then hit TAB to get the character $\beta$. 


In [ ]:
β=[1,2,3]
5*β

List of unicode characters supported by Julia can be found in Julia documentation. Codig in Julia may be visually very interesting. For instance, you can produce the following code to count cups of beer. 

In [ ]:
🍺=1
🍻=2
🍺+🍻

By  the standard Euclidean inner (dot) product of two vectors $\mathbf{u} = (u_1,\dots,u_n)$ and $\mathbf{v}=(v_1,\dots,v_n)$ from $\mathbb{R}^n$ we mean the quantity  $$\langle \mathbf{u}, \mathbf{v} \rangle = \mathbf{u} \cdot \mathbf{v}= \mathbf{u}^\top \mathbf{v} = \sum_{i=1}^n u_i v_i.$$ 
All of the above notations are used in specific contexts in various fields, which can be confusing.

In order to compute this in Julia, you will most often want to do this via the operation <b>u $\cdot$ v</b> than via <b>u'\*v</b>. Both are valid Julia expressions for computing the inner product. However, while the output of <b>u $\cdot$ v</b> is a scalar value, the output of <b>u'*v</b> is a $1\times 1$ matrix. Be aware of this, as it can break your code sometimes.

In [3]:
v = [1,4,5]
u = [2,6,1]
print(u⋅v)

print(u'*v)

LoadError: syntax: invalid character "·"

## 2. Gradient descent
The goal of this lab is to form better intuition about how gradient descent works.  You can find some basic information in  slides from Lecture 1. 

Throughout the rest of the lab we would like to solve the following least-squares problem:
$$\mathrm{min}\ f(x):=\frac12 \|Ax-b\|^2 .$$

This is an important prboblem to solve on its own, with a plethora of applications. At the same time, it isa  prototype of a convex optimization problem, and one can get understanding by seeing how various optimization algorithms work on this problem.

Let $x^0$ be a starting point,  $\{\alpha^t\}>0$ a sequence of stepsize parameters. Gradient descent (GD) is given by the following recursion:
$$x^{t+1}=x^t -\alpha^t \nabla f(x^t).$$


In [ ]:
function f(x,A,b)
    return norm(A*x-b)/2
end

In [ ]:
A=reshape([0,1,1,0],2,2)
x=[1,2]
b=[5,5]
f(x,A,b)

It will be useful to have access to the gradient of $f$. **Complete the function in the cell below.**

In [ ]:
function gradf(x,A,b)
    return A'*(A*x-b)
end

Now, **write a code for Gradient Descent in the cell below**. It should return a matrix having the starting point as the first row, first iterate as the second row, ..., $k$-th iterate as the $(k+1)$-th row. 

In [ ]:
function GradientDescent(∇f,A,b,α,x0,k)
#α - vector of stepsizes, α[t]=α^t
#x0 - starting point
#k - number if iterations
    x=ones(k+1,size(x0)[1])
    x[1,:]=x0
    for i=1:k
        x[i+1,:]=x[i,:]-α[i]*∇f(x[i,:],A,b)
    end
    return x
end

In [ ]:
using PyPlot
function plot_decrease(x,x_star,A,b,f,k,col)
    fs = zeros(k)
    for t=1:k
        fs[t] = f(x[t,:],A,b)
    end
    f_star=f(x_star,A,b)
    semilogy(1:k, fs-f_star, color=col) # plot on graph with logarithmic y-axis
    xlabel(L"t")
    ylabel(L"f(x^{(t)})-f(x^*)")
    title("Convergence of the functional value")
    grid("on")
end

### 2. Testing Gradient Descent on a simple problem

Let's put everything together and run the gradient descent for some simple choice of $A,\, b$ to check whether it actually works. Try to run it for **different choices of $\alpha$** to see how the stepsize choice influences the convergence speed.

In [ ]:
# form a 3-by-3 diagonal matrix with values 2, 1, 3 on the diagonal
A=diagm([2,1,3])

b=[2,1,3]

# it is easy to see that the solution of the optimization problem is x=[1,1,1]
x_star=[1,1,1]

# setting the starting point
x0=[10,3,8]
 
# number of iterations
k=100

#Setting stepsize. Should be less than 1/λ_max(A'*A)
α=0.01*ones(1,k)


x=GradientDescent(gradf,A,b,α,x0,k)

plot_decrease(x,x_star,A,b,f,k,"blue")

## 3. Testing Gradient Descent on a more complicated problem

Now let's run gradient descent on some more complicated problem. 

We are going to generate a random matrix $A$. We would like to have $A^{\top}A$ (hessian of $f$) to be positive definite and have the control of its condition number, i.e. 
$$\frac{\lambda_{\mathrm{max}}(A^{\top}A)}{\lambda_{\mathrm{min}}(A^{\top}A)}=
\frac{\sigma_{\mathrm{max}}(A)}{\sigma_{\mathrm{min}}(A)}$$
From singular value decomposition, we will generate $A$ as product $UDV$ where $U,\, V$ are orthogonal matrices and $D=\mathrm{diag}(\sigma_1,\dots, \sigma_n)$. 

**Play with the condition number $\kappa$ and stepsize parameter $\alpha$ to see how they influence the convergence of the algorithm. **


In [ ]:
#dimension of the problem
d=3
#conditional numbar of hessian. Play with it and see what happens.  
κ=10
#minimal eigenvalue of the hessian
λ_min=1

#generate U,V as orthogonal matrices
U=qr(rand(d,d))[1]
V=qr(rand(d,d))[1]

#generate D 
vec_D=vec(vcat([sqrt(λ_min*κ)],sqrt(λ_min).+rand(d-2,1)*(sqrt(λ_min*κ)-sqrt(λ_min)),[sqrt(λ_min)]))
D=diagm(vec_D)

#get A from SVD
A=U*D*V

#chceck condition number of hessian (is should be close to κ)
println("cond(A'*A)= ",cond(A'*A))

#random initialization of b
b=randn(d,1)

#random initialization of x0
x0=randn(d,1)

# number of iterations
k=100

# α should be less than 1/λ_max(A'*A). Play with it and see what happens 
α=0.01*ones(1,k)

#running gradient descent
x=GradientDescent(gradf,A,b,α,x0,k)

#getting the exact solution
x_star=\(A,b)

#plotting the results
plot_decrease(x,x_star,A,b,f,k,"blue")


## 4. Projected Gradient Descent
Projected Gradient Descent is used to minimize function $f$ over a set $Q \in \mathbb{R}^n$. The iterations are given by

$$ x^{t+1}=\Pi_Q\big(x^t-\alpha^t\nabla f(x^t)\big), \qquad \Pi_Q(y)=\mathrm{argmin}_{z\in Q}\|y-z \|^2 $$
    
Above, $\Pi_Q(y)$ is the Euclidean projection of $y$ onto the set $Q$.
 
Your task is to **write the code of Projected Gradient Descent  in the cell below**. Use the constraint  $Q=\{x \;:\; \|x\|^2=1 \}$ (unit ball). First, you will need to figure out on a piece of paper what the formula for $\Pi_{Q}(y)$ is. 

If you are fast with this, play with some other constraints, such as $Q = \{x\;:\; a^\top x = 1\}$ for some vector $a\in \mathbb{R}^n$.

As before, the code should return a matrix having the starting point as the first row, first iterate as the second row, ..., $k$-th iterate as the $(k+1)$-th row.

In [ ]:
function ProjGD(∇f,A,b,α,x0,k)
#α - vector of stepsizes, α[t]=α^t
#x0 - starting point
#k - number if iterations
    x=ones(k+1,size(A)[1])
    x[1,:]=x0
    for i=1:k
        xc=x[i,:]-α[i]*∇f(x[i,:],A,b)
        x[i+1,:]=xc/(norm(xc))
    end
    return x
end

Now we have implemented all neccessary tools to test the Projected Gradient Method. Try to run the code below for **different choices of $\alpha$** to see how the stepsizes influence the convergence speed.

In [ ]:
A=eye(3)
b=[4,4,4]
#Note that the solution is x=[1,1,1]
x_star=[ 1/sqrt(3), 1/sqrt(3), 1/sqrt(3)]

#setting the starting point
x0=[-1/sqrt(2),-1/sqrt(2),0]
 
#number of iterations
k=30

##Setting stepsize. Good choice can be 1/λ_max(A'*A)
α=100*ones(1,k)

x=ProjGD(gradf,A,b,α,x0,k)

plot_decrease(x,x_star,A,b,f,k,"blue")

## 5. Accelerated Gradient Descent

Accelerated Gradient Descent (AGD) is an very fast method for minimizing a smooth and strongly convex function.  
The iterations are given recursively by
    $$y^{t+1}=x^t-\alpha^t\nabla f(x^t), \qquad x^{k+1}=(1+\beta^t)y^{t+1}-\beta^ty^t .$$
 
Your task is to **write the code of Accelerated Gradient Descent algorithm** in the cell below. It should return a matrix having the starting point as the first row, first iterate as the second row, ..., $k$-th iterate as the $(k+1)$-th row.


In [ ]:
function AccGD(∇f,A,b,α,β,x0,k)
#α - stepsize parameter vector
#β - stepsize parameter vector
#x0 - starting point
#k - number if iterations
    x=ones(k+1,size(x0)[1])
    x[1,:]=x0
    y=ones(k+1,size(x0)[1])
    y[1,:]=x0
    for i=1:k
        y[i+1,:]=x[i,:]-α[i]*∇f(x[i,:],A,b)
        x[i+1,:]=(1+β[i])*y[i+1,:]-β[i]*y[i,:] 
    end
    return x
end

Try to run the code below for **different choices of $\alpha$, $\beta$ and $\kappa$** to to see how the stepsize and condition number influence the convergence speed.

In [ ]:
#dimension of the problem
d=100
#conditional numbar of hessian. Play with it and see what happens.  
κ=50

#minimal eigenvalue of the hessian
λ_min=1

#getting A from SVD decomposition
#generate U,V as orthogonal matrices
U=qr(rand(d,d))[1]
V=qr(rand(d,d))[1]

#generate D 
vec_D=vec(vcat([sqrt(λ_min*κ)],sqrt(λ_min).+rand(d-2,1)*(sqrt(λ_min*κ)-sqrt(λ_min)),[sqrt(λ_min)]))
D=diagm(vec_D)


#get A from SVD
A=U*D*V

#chceck condition number of hessian 
println("cond(A'*A)= ",cond(A'*A))

#random initialization of b
b=randn(d,1)

#random initialization of x0
x0=randn(d,1)

# number of iterations
k=1000

# Play with α to get a better intuition about the algorithm. Good choice can be 1/(κ*λ_min)*ones(1,n)
α=0.01*ones(1,k)
# Play with β to get a better intuition about the algorithm. Good choice can be (sqrt(κ)-1)/(sqrt(κ+1))*ones(1,n)
β=0.5*(sqrt(κ)-1)/(sqrt(κ+1))*ones(1,k)


#running accelerated gradient descent
x=AccGD(gradf,A,b,α,β,x0,k)

#getting the exact solution
x_star=\(A,b)

#plotting the results
plot_decrease(x,x_star,A,b,f,k,"blue")

#comparing with gradient descent
x_gd=GradientDescent(gradf,A,b,α,x0,k)
plot_decrease(x_gd,x_star,A,b,f,k,"red")
